# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [16]:
# 1) Seu código aqui
request_RJ = requests.get(
   'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio')
request_RJ.status_code

df = pd.DataFrame(request_RJ.json()['items'])
df.head()



,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,65517,Município,Nova Iguaçu,3303500,RJ,PVL02.000206/2023-14,Deferido,17944.102874/2023-48,2023-07-25T14:16:18Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado do Rio de Janeiro,Real,5.000000e+07,0,1,28/07/2023
1,11272,Município,Rio de Janeiro,3304557,RJ,None,Deferido,17944.001398/2013-77,2013-11-25T00:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5.000000e+08,1,0,06/12/2013
2,11202,Município,Rio de Janeiro,3304557,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001383/2012-28,2013-07-19T00:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.620000e+07,1,0,13/08/2013
3,12602,Município,Vassouras,3306206,RJ,None,Arquivado,17944.001723/2008-34,2008-07-10T00:00:00Z,Operação contratual interna,Caminho da Escola,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.029900e+06,0,0,29/07/2008
4,12404,Município,Resende,3304201,RJ,None,Regularizado,17944.001679/2008-62,2013-07-12T00:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Ampla Energia e Serviços S/A,Real,4.397135e+06,1,0,26/07/2013


In [18]:
# 2) Seu código aqui
df['uf'].value_counts()


uf
RJ    268
Name: count, dtype: int64

In [38]:
# 3) Seu código aqui
df['ano_status'] =  df['data_status'].str.slice(start=6)
ano_status

0      2023
1      2013
2      2013
3      2008
4      2013
       ... 
263    2007
264    2012
265    2019
266    2014
267    2013
Name: data_status, Length: 268, dtype: object

In [46]:
anos = df['ano_status'].value_counts()
print("Frequências dos anos:")
print(anos)

Frequências dos anos:
ano_status
2008    37
2007    29
2023    28
2014    20
2010    19
2019    16
2020    15
2013    15
2012    13
2022    11
2024    10
2011    10
2009     9
2015     8
2017     5
2021     5
2016     4
2018     4
2002     4
2006     4
2004     1
2003     1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [100]:
#1) Seu código aqui
def Consulta_Api (uf, tipo_interessado):
    url = f"https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}"
    response = requests.get(url)

    if response.status_code == 200:
         dados = response.json().get('items')
         return pd.DataFrame(dados)

    else:
        print('Erro ao acessar a API, tente novamente')

In [102]:
# 2) Seu código aqui
df_mg = Consulta_Api('MG', 'Estado')

solicitacoes_arquivadas = df_mg[df_mg['status'] == 'Arquivado por decurso de prazo']

quantidade = len(solicitacoes_arquivadas)
print(f"Quantidade de solicitações arquivadas por decurso de prazo em MG: {quantidade}")

Quantidade de solicitações arquivadas por decurso de prazo em MG: 1


In [108]:
df_ba = Consulta_Api('BA', 'Município')
df_ba.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,42393,Município,Rafael Jambeiro,2925956,BA,PVL02.000961/2020-56,Deferido,17944.101782/2020-06,2020-04-06T11:57:42Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,3000000.0,0,0,13/04/2020
1,41920,Município,Irecê,2914604,BA,PVL02.000567/2020-18,Deferido,17944.100792/2020-16,2020-02-27T17:05:26Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,7755555.0,0,1,03/03/2020
2,65767,Município,Dom Basílio,2910107,BA,PVL02.002821/2023-65,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1055900.0,0,0,27/09/2023
3,67099,Município,Jiquiriçá,2918209,BA,PVL02.004047/2023-27,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4800000.0,0,1,07/11/2023
4,66798,Município,Urandi,2932606,BA,PVL02.003766/2023-21,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,3000000.0,0,0,27/09/2023


In [110]:
# 3) Seu código aqui
df_ba = Consulta_Api('BA', 'Município')
df_ba.head()

solicitacoes_deferidas = df_ba[df_ba['status'] == 'Deferido']

municipio_mais_deferidas = solicitacoes_deferidas['interessado'].value_counts().idxmax()
print(municipio_mais_deferidas)

Luís Eduardo Magalhães


Empty DataFrame
Columns: []
Index: []
RangeIndex(start=0, stop=0, step=1)


In [114]:
# 4) Seu código aqui
df_ba_estado = Consulta_Api('BA', 'Estado')
df_ba_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)
